In [1]:
from nltk.corpus import treebank
import nltk
from nltk import pos_tag, word_tokenize
from nltk.tag import CRFTagger

In [2]:
ct = CRFTagger()

In [6]:
train_sentences = treebank.tagged_sents()[:3000]
test_sentences = treebank.tagged_sents()[3000:]